In [5]:
! pip install tqdm
! pip install joblib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [6]:
# Config
import os

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob

# Parallel processing
from joblib import Parallel, delayed

import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO

In [2]:
configs = ["obb_base_y8x", "obb_base_y11l", "obb_base_y11m", "obb_base", "obb_base_y8l", "obb_base_y8m"]
config_to_model_mapping = {}
for config in configs:
    with open(f"/home/patel_zeel/kiln_compass_24/configs/{config}.py") as f:
        config_content = f.read()
    model_name = re.search(r"model = \"([^']+?)\"", config_content).group(1)
    config_to_model_mapping[config] = model_name.replace(".pt", "")

print(config_to_model_mapping)

data_yml_save_dir ="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/yaml_data_dir/"
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_save_path = join(data_yml_save_dir, "data.yml")
with open(data_yml_save_path, "w") as f:
    f.write(data_yml)

image_dir = "/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/"
gt_dir = "/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/labels/"
sv_dataset = sv.DetectionDataset.from_yolo(image_dir, gt_dir, data_yml_save_path,is_obb=True)
print(sv_dataset)
len(sv_dataset)

{'obb_base_y8x': 'yolov8x-obb', 'obb_base_y11l': 'yolo11l-obb', 'obb_base_y11m': 'yolo11m-obb', 'obb_base': 'yolo11x-obb', 'obb_base_y8l': 'yolov8l-obb', 'obb_base_y8m': 'yolov8m-obb'}


84

In [8]:
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"])

for config in configs:
    model_name = config_to_model_mapping[config]
    model = YOLO(f"/home/patel_zeel/kiln_compass_24/runs/obb/config_name={config}&train_folder=m0_obb_without_empty_train&val_folder=m0_obb_without_empty_val/weights/best.pt")
    results_dir = f"/home/patel_zeel/kiln_compass_24/runs/obb/config_name={config}&train_folder=m0_obb_without_empty_train&val_folder=m0_obb_without_empty_val&z_predict_region=m0_obb_without_empty_val"
    predictions_dir = join(results_dir, "labels")
    
    targets = []
    predictions = []
    for name, _, gt_detection in tqdm(sv_dataset):
        print(name)
        print(gt_detection)
        file_name = splitext(basename(name))[0]
        prediction_path = join(predictions_dir, f"{file_name}.txt")
        ultralytics_result = model(name, imgsz=640, iou=0.33, max_det=300, conf=0.001, verbose=False)[0]
        sv_detections = sv.Detections.from_ultralytics(ultralytics_result)
        targets.append(gt_detection)
        predictions.append(sv_detections)
    
    mAP_metric = MeanAveragePrecision(class_agnostic=False)
    mAP_result = mAP_metric.update(predictions, targets).compute()
    class_wise_result = mAP_result.ap_per_class[:, 0].tolist()
    
    mAP_metric = MeanAveragePrecision(class_agnostic=True)
    mAP_result = mAP_metric.update(predictions, targets).compute()
    class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
    result_df.loc[model_name, :] = class_wise_result + [mAP_result.mAP_scores[0].item()] + class_agnostic_result

  0%|          | 0/84 [00:00<?, ?it/s]

/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/9021406_3115876.tif
Detections(xyxy=array([[     588.72,      598.33,      611.97,      619.45],
       [     412.18,      500.21,      435.18,      525.91],
       [     556.31,      494.41,      574.25,      515.69],
       [     116.37,      421.41,      139.76,      447.16],
       [     312.36,      366.31,      336.87,       383.9],
       [     470.07,      328.57,      494.09,      357.35],
       [     77.549,       320.1,      102.44,       340.8],
       [     508.96,      209.33,      530.53,       235.6],
       [     414.61,      171.45,      446.39,      198.56],
       [     215.84,      137.34,       241.9,       162.3],
       [     139.97,      37.537,      161.02,      63.442],
       [     33.789,      17.958,      67.436,      45.091]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, Fal

  0%|          | 0/84 [00:00<?, ?it/s]

/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/9021406_3115876.tif
Detections(xyxy=array([[     588.72,      598.33,      611.97,      619.45],
       [     412.18,      500.21,      435.18,      525.91],
       [     556.31,      494.41,      574.25,      515.69],
       [     116.37,      421.41,      139.76,      447.16],
       [     312.36,      366.31,      336.87,       383.9],
       [     470.07,      328.57,      494.09,      357.35],
       [     77.549,       320.1,      102.44,       340.8],
       [     508.96,      209.33,      530.53,       235.6],
       [     414.61,      171.45,      446.39,      198.56],
       [     215.84,      137.34,       241.9,       162.3],
       [     139.97,      37.537,      161.02,      63.442],
       [     33.789,      17.958,      67.436,      45.091]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, Fal

  0%|          | 0/84 [00:00<?, ?it/s]

...

/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/9021406_3115876.tif
Detections(xyxy=array([[     588.72,      598.33,      611.97,      619.45],
       [     412.18,      500.21,      435.18,      525.91],
       [     556.31,      494.41,      574.25,      515.69],
       [     116.37,      421.41,      139.76,      447.16],
       [     312.36,      366.31,      336.87,       383.9],
       [     470.07,      328.57,      494.09,      357.35],
       [     77.549,       320.1,      102.44,       340.8],
       [     508.96,      209.33,      530.53,       235.6],
       [     414.61,      171.45,      446.39,      198.56],
       [     215.84,      137.34,       241.9,       162.3],
       [     139.97,      37.537,      161.02,      63.442],
       [     33.789,      17.958,      67.436,      45.091]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, Fal

  0%|          | 0/84 [00:00<?, ?it/s]

/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/9021406_3115876.tif
Detections(xyxy=array([[     588.72,      598.33,      611.97,      619.45],
       [     412.18,      500.21,      435.18,      525.91],
       [     556.31,      494.41,      574.25,      515.69],
       [     116.37,      421.41,      139.76,      447.16],
       [     312.36,      366.31,      336.87,       383.9],
       [     470.07,      328.57,      494.09,      357.35],
       [     77.549,       320.1,      102.44,       340.8],
       [     508.96,      209.33,      530.53,       235.6],
       [     414.61,      171.45,      446.39,      198.56],
       [     215.84,      137.34,       241.9,       162.3],
       [     139.97,      37.537,      161.02,      63.442],
       [     33.789,      17.958,      67.436,      45.091]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, Fal

  0%|          | 0/84 [00:00<?, ?it/s]

/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images/9021406_3115876.tif
Detections(xyxy=array([[     588.72,      598.33,      611.97,      619.45],
       [     412.18,      500.21,      435.18,      525.91],
       [     556.31,      494.41,      574.25,      515.69],
       [     116.37,      421.41,      139.76,      447.16],
       [     312.36,      366.31,      336.87,       383.9],
       [     470.07,      328.57,      494.09,      357.35],
       [     77.549,       320.1,      102.44,       340.8],
       [     508.96,      209.33,      530.53,       235.6],
       [     414.61,      171.45,      446.39,      198.56],
       [     215.84,      137.34,       241.9,       162.3],
       [     139.97,      37.537,      161.02,      63.442],
       [     33.789,      17.958,      67.436,      45.091]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, Fal